In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [8]:
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE

ada= SMOTE(random_state= 15)
cc= TomekLinks()

X_ada, y_ada= ada.fit_resample(X_train, y_train)
X_ada_cc, y_ada_cc= cc.fit_resample(X_ada, y_ada)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr_ada_cc= LogisticRegression(random_state=15)
lr_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_lr_ada_cc= lr_ada_cc.predict(X_test)
y_pred_prob_lr_ada_cc= lr_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_ada_cc, labels= lr_ada_cc.classes_)
print(classification_report(y_test, y_pred_lr_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_lr_ada_cc))

              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.26      0.81      0.39       912

    accuracy                           0.76      9429
   macro avg       0.61      0.78      0.62      9429
weighted avg       0.90      0.76      0.80      9429

Precision: 0.25636553889082664
Accuracy: 0.7551171916428041
AUC: 0.7777990844935516


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_ada_cc= DecisionTreeClassifier(random_state=15)
dt_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_dt_ada_cc= dt_ada_cc.predict(X_test)
y_pred_prob_dt_ada_cc= dt_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_ada_cc, labels= dt_ada_cc.classes_)
print(classification_report(y_test, y_pred_dt_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_ada_cc))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      8517
           1       0.24      0.28      0.26       912

    accuracy                           0.84      9429
   macro avg       0.58      0.59      0.58      9429
weighted avg       0.86      0.84      0.85      9429

Precision: 0.23716153127917833
Accuracy: 0.8435677166189416
AUC: 0.59074086089946


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_ada_cc= GridSearchCV(estimator= dt_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_dt_ada_cc.best_params_)
print("Best Precision Score:", gs_dt_ada_cc.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 10, 'min_samples_split': 2}
Best Precision Score: 0.9414767107948647


In [12]:
tuned_dt_ada_cc= gs_dt_ada_cc.best_estimator_
y_pred_tuned_dt_ada_cc= tuned_dt_ada_cc.predict(X_test)
y_pred_prob_tuned_dt_ada_cc= tuned_dt_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_ada_cc, labels= tuned_dt_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_dt_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_ada_cc))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      8517
           1       0.31      0.23      0.26       912

    accuracy                           0.88      9429
   macro avg       0.61      0.59      0.60      9429
weighted avg       0.86      0.88      0.87      9429

Precision: 0.30612244897959184
Accuracy: 0.8750662848658395
AUC: 0.7174629070033308


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_ada_cc= KNeighborsClassifier()
knn_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_knn_ada_cc= knn_ada_cc.predict(X_test)
y_pred_prob_knn_ada_cc= knn_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_ada_cc, labels= knn_ada_cc.classes_)
print(classification_report(y_test, y_pred_knn_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_ada_cc))

              precision    recall  f1-score   support

           0       0.95      0.78      0.85      8517
           1       0.22      0.58      0.32       912

    accuracy                           0.76      9429
   macro avg       0.58      0.68      0.59      9429
weighted avg       0.88      0.76      0.80      9429

Precision: 0.218878812860676
Accuracy: 0.758617032559126
AUC: 0.7388068934370682


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_ada_cc= GridSearchCV(estimator= knn_ada_cc, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_ada_cc.fit(X_ada_cc, y_ada_cc)
print("Best Parameters:", gs_knn_ada_cc.best_params_)
print("Best Precision Score:", gs_knn_ada_cc.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.9307749431395962


In [15]:
tuned_knn_ada_cc= gs_knn_ada_cc.best_estimator_
y_pred_tuned_knn_ada_cc= tuned_knn_ada_cc.predict(X_test)
y_pred_prob_tuned_knn_ada_cc= tuned_knn_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_ada_cc, labels= tuned_knn_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_knn_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_ada_cc))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      8517
           1       0.25      0.19      0.22       912

    accuracy                           0.87      9429
   macro avg       0.58      0.57      0.57      9429
weighted avg       0.85      0.87      0.86      9429

Precision: 0.25
Accuracy: 0.8657333757556475
AUC: 0.6209813345445333


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_ada_cc= GaussianNB()
nb_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_nb_ada_cc= nb_ada_cc.predict(X_test)
y_pred_prob_nb_ada_cc= nb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_ada_cc, labels= nb_ada_cc.classes_)
print(classification_report(y_test, y_pred_nb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_ada_cc))

              precision    recall  f1-score   support

           0       0.97      0.55      0.70      8517
           1       0.17      0.86      0.28       912

    accuracy                           0.58      9429
   macro avg       0.57      0.70      0.49      9429
weighted avg       0.90      0.58      0.66      9429

Precision: 0.1689291101055807
Accuracy: 0.5773676954077845
AUC: 0.7983264636876917


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_ada_cc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_svc_ada_cc= svc_ada_cc.predict(X_test)
y_pred_prob_svc_ada_cc= svc_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_ada_cc, labels= svc_ada_cc.classes_)
print(classification_report(y_test, y_pred_svc_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_ada_cc))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      8517
           1       0.15      0.03      0.06       912

    accuracy                           0.89      9429
   macro avg       0.53      0.51      0.50      9429
weighted avg       0.83      0.89      0.85      9429

Precision: 0.15121951219512195
Accuracy: 0.8881111464630396
AUC: 0.6975077193394428


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_ada_cc= RandomForestClassifier(random_state=15)
rf_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_rf_ada_cc= rf_ada_cc.predict(X_test)
y_pred_prob_rf_ada_cc= rf_ada_cc.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_ada_cc, labels= rf_ada_cc.classes_)
print(classification_report(y_test, y_pred_rf_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_ada_cc))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.41      0.14      0.21       912

    accuracy                           0.90      9429
   macro avg       0.66      0.56      0.58      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.41233766233766234
Accuracy: 0.8975501113585747
AUC: 0.8210167642012158


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_ada_cc= GridSearchCV(estimator= rf_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_ada_cc.fit(X_ada_cc, y_ada_cc)
print("Best Parameters:", gs_tuned_rf_ada_cc.best_params_)
print("Best Precision Score:", gs_tuned_rf_ada_cc.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Best Precision Score: 0.9745044500897784


In [21]:
tuned_rf_ada_cc= gs_tuned_rf_ada_cc.best_estimator_
y_pred_tuned_rf_ada_cc= tuned_rf_ada_cc.predict(X_test)
y_pred_prob_tuned_rf_ada_cc= tuned_rf_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_ada_cc, labels= tuned_rf_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_rf_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_ada_cc))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.44      0.14      0.21       912

    accuracy                           0.90      9429
   macro avg       0.68      0.56      0.58      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.4412811387900356
Accuracy: 0.8997772828507795
AUC: 0.8331024354799174


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_ada_cc= AdaBoostClassifier(random_state=15)
ada_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_ada_ada_cc= ada_ada_cc.predict(X_test)
y_pred_prob_ada_ada_cc= ada_ada_cc.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_ada_cc, labels= ada_ada_cc.classes_)
print(classification_report(y_test, y_pred_ada_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_ada_cc))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93      8517
           1       0.38      0.44      0.41       912

    accuracy                           0.88      9429
   macro avg       0.66      0.68      0.67      9429
weighted avg       0.88      0.88      0.88      9429

Precision: 0.3771266540642722
Accuracy: 0.875702619577898
AUC: 0.8427933863954238


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_ada_cc= GridSearchCV(estimator= ada_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_ada_cc.fit(X_ada_cc, y_ada_cc)
print("Best Parameters:", gs_ada_ada_cc.best_params_)
print("Best Precision Score:", gs_ada_ada_cc.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9623657381206303


In [25]:
tuned_ada_ada_cc= gs_ada_ada_cc.best_estimator_
y_pred_tuned_ada_ada_cc= tuned_ada_ada_cc.predict(X_test)
y_pred_prob_tuned_ada_ada_cc= tuned_ada_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_ada_cc, labels= tuned_ada_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_ada_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_ada_cc))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95      8517
           1       0.47      0.27      0.34       912

    accuracy                           0.90      9429
   macro avg       0.70      0.62      0.64      9429
weighted avg       0.88      0.90      0.89      9429

Precision: 0.47126436781609193
Accuracy: 0.9000954502068088
AUC: 0.8475041017037134


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_ada_cc= GradientBoostingClassifier(random_state=15)
grb_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_grb_ada_cc= grb_ada_cc.predict(X_test)
y_pred_prob_grb_ada_cc= grb_ada_cc.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_ada_cc, labels= grb_ada_cc.classes_)
print(classification_report(y_test, y_pred_grb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_ada_cc))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      8517
           1       0.41      0.32      0.36       912

    accuracy                           0.89      9429
   macro avg       0.67      0.64      0.65      9429
weighted avg       0.88      0.89      0.88      9429

Precision: 0.4133522727272727
Accuracy: 0.8903383179552444
AUC: 0.8444858219577357


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_ada_cc= GridSearchCV(estimator= grb_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_grb_ada_cc.best_params_)
print("Best Precision Score:", gs_grb_ada_cc.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.9720556457518429


In [28]:
tuned_grb_ada_cc= gs_grb_ada_cc.best_estimator_
y_pred_tuned_grb_ada_cc= tuned_grb_ada_cc.predict(X_test)
y_pred_prob_tuned_grb_ada_cc= tuned_grb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_ada_cc, labels= tuned_grb_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_grb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_ada_cc))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.48      0.19      0.27       912

    accuracy                           0.90      9429
   macro avg       0.70      0.59      0.61      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.48476454293628807
Accuracy: 0.9021105101283275
AUC: 0.8467269537292803


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_ada_cc= XGBClassifier(random_state=15)
xgb_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_xgb_ada_cc= xgb_ada_cc.predict(X_test)
y_pred_prob_xgb_ada_cc= xgb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_ada_cc, labels= xgb_ada_cc.classes_)
print(classification_report(y_test, y_pred_xgb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_ada_cc))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.46      0.14      0.21       912

    accuracy                           0.90      9429
   macro avg       0.69      0.56      0.58      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.46096654275092935
Accuracy: 0.9010499522748966
AUC: 0.8320702506236238


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_ada_cc= GridSearchCV(estimator= xgb_ada_cc, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_xgb_ada_cc.best_params_)
print("Best Precision Score:", gs_xgb_ada_cc.best_score_)

tuned_xgb_ada_cc= gs_xgb_ada_cc.best_estimator_
y_pred_tuned_xgb_ada_cc= tuned_xgb_ada_cc.predict(X_test)
y_pred_prob_tuned_xgb_ada_cc= tuned_xgb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_ada_cc, labels= tuned_xgb_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_ada_cc))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.9684046430716761
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      8517
           1       0.47      0.22      0.30       912

    accuracy                           0.90      9429
   macro avg       0.70      0.60      0.62      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.47368421052631576
Accuracy: 0.9009438964895535
AUC: 0.8436872385260439


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_ada_cc= LGBMClassifier(random_state=15)
lgm_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_lgm_ada_cc= lgm_ada_cc.predict(X_test)
y_pred_prob_lgm_ada_cc= lgm_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_ada_cc, labels= lgm_ada_cc.classes_)
print(classification_report(y_test, y_pred_lgm_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_ada_cc))

[LightGBM] [Info] Number of positive: 33993, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009778 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8572
[LightGBM] [Info] Number of data points in the train set: 68058, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499471 -> initscore=-0.002116
[LightGBM] [Info] Start training from score -0.002116
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.50      0.15      0.23       912

    accuracy                           0.90      9429
   macro avg       0.71      0.57      0.59      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5036496350364964
Accuracy: 0.9034892353377877
AUC: 0.8460923869495273


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_ada_cc= GridSearchCV(estimator= lgm_ada_cc, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_lgm_ada_cc.best_params_)
print("Best Precision Score:", gs_lgm_ada_cc.best_score_)

tuned_lgm_ada_cc= gs_lgm_ada_cc.best_estimator_
y_pred_tuned_lgm_ada_cc= tuned_lgm_ada_cc.predict(X_test)
y_pred_prob_tuned_lgm_ada_cc= tuned_lgm_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_ada_cc, labels= tuned_lgm_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_ada_cc))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010127 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8572
[LightGBM] [Info] Number of data points in the train set: 68058, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499471 -> initscore=-0.002116
[LightGBM] [Info] Start training from score -0.002116
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 300, 'num_leaves': 31, 'reg_alpha': 0.5}
Best Precision Score: 0.9799990

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_ada_cc= CatBoostClassifier(random_state=15)
cat_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_cat_ada_cc= cat_ada_cc.predict(X_test)
y_pred_prob_cat_ada_cc= cat_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_ada_cc, labels= cat_ada_cc.classes_)
print(classification_report(y_test, y_pred_cat_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_ada_cc))

Learning rate set to 0.062456
0:	learn: 0.6253801	total: 190ms	remaining: 3m 9s
1:	learn: 0.5697059	total: 214ms	remaining: 1m 47s
2:	learn: 0.5272632	total: 236ms	remaining: 1m 18s
3:	learn: 0.4940724	total: 255ms	remaining: 1m 3s
4:	learn: 0.4647067	total: 274ms	remaining: 54.5s
5:	learn: 0.4336111	total: 293ms	remaining: 48.6s
6:	learn: 0.4070101	total: 313ms	remaining: 44.3s
7:	learn: 0.3924567	total: 332ms	remaining: 41.2s
8:	learn: 0.3787377	total: 352ms	remaining: 38.7s
9:	learn: 0.3663653	total: 371ms	remaining: 36.7s
10:	learn: 0.3495764	total: 392ms	remaining: 35.3s
11:	learn: 0.3370465	total: 412ms	remaining: 33.9s
12:	learn: 0.3276513	total: 432ms	remaining: 32.8s
13:	learn: 0.3145927	total: 452ms	remaining: 31.8s
14:	learn: 0.3071404	total: 471ms	remaining: 30.9s
15:	learn: 0.3001535	total: 490ms	remaining: 30.2s
16:	learn: 0.2882476	total: 513ms	remaining: 29.6s
17:	learn: 0.2822027	total: 532ms	remaining: 29s
18:	learn: 0.2731056	total: 554ms	remaining: 28.6s
19:	learn: 

163:	learn: 0.1533834	total: 3.88s	remaining: 19.8s
164:	learn: 0.1532973	total: 3.9s	remaining: 19.8s
165:	learn: 0.1530714	total: 3.93s	remaining: 19.7s
166:	learn: 0.1529874	total: 3.95s	remaining: 19.7s
167:	learn: 0.1528986	total: 3.97s	remaining: 19.7s
168:	learn: 0.1527954	total: 3.99s	remaining: 19.6s
169:	learn: 0.1526914	total: 4.01s	remaining: 19.6s
170:	learn: 0.1525540	total: 4.04s	remaining: 19.6s
171:	learn: 0.1524162	total: 4.06s	remaining: 19.5s
172:	learn: 0.1522780	total: 4.08s	remaining: 19.5s
173:	learn: 0.1522128	total: 4.1s	remaining: 19.5s
174:	learn: 0.1518986	total: 4.12s	remaining: 19.4s
175:	learn: 0.1518295	total: 4.14s	remaining: 19.4s
176:	learn: 0.1517671	total: 4.17s	remaining: 19.4s
177:	learn: 0.1516940	total: 4.19s	remaining: 19.4s
178:	learn: 0.1512340	total: 4.21s	remaining: 19.3s
179:	learn: 0.1504568	total: 4.24s	remaining: 19.3s
180:	learn: 0.1503692	total: 4.26s	remaining: 19.3s
181:	learn: 0.1502828	total: 4.28s	remaining: 19.3s
182:	learn: 0.

322:	learn: 0.1352559	total: 7.47s	remaining: 15.7s
323:	learn: 0.1351650	total: 7.49s	remaining: 15.6s
324:	learn: 0.1350996	total: 7.51s	remaining: 15.6s
325:	learn: 0.1350280	total: 7.54s	remaining: 15.6s
326:	learn: 0.1349847	total: 7.56s	remaining: 15.6s
327:	learn: 0.1349203	total: 7.58s	remaining: 15.5s
328:	learn: 0.1348719	total: 7.61s	remaining: 15.5s
329:	learn: 0.1348298	total: 7.63s	remaining: 15.5s
330:	learn: 0.1347642	total: 7.66s	remaining: 15.5s
331:	learn: 0.1346947	total: 7.68s	remaining: 15.5s
332:	learn: 0.1346152	total: 7.7s	remaining: 15.4s
333:	learn: 0.1345602	total: 7.73s	remaining: 15.4s
334:	learn: 0.1345126	total: 7.75s	remaining: 15.4s
335:	learn: 0.1344651	total: 7.77s	remaining: 15.4s
336:	learn: 0.1344217	total: 7.79s	remaining: 15.3s
337:	learn: 0.1343877	total: 7.81s	remaining: 15.3s
338:	learn: 0.1343417	total: 7.83s	remaining: 15.3s
339:	learn: 0.1342999	total: 7.85s	remaining: 15.2s
340:	learn: 0.1341768	total: 7.88s	remaining: 15.2s
341:	learn: 0

488:	learn: 0.1259263	total: 11.3s	remaining: 11.8s
489:	learn: 0.1258839	total: 11.3s	remaining: 11.7s
490:	learn: 0.1257973	total: 11.3s	remaining: 11.7s
491:	learn: 0.1257053	total: 11.3s	remaining: 11.7s
492:	learn: 0.1256217	total: 11.3s	remaining: 11.7s
493:	learn: 0.1255768	total: 11.4s	remaining: 11.6s
494:	learn: 0.1255421	total: 11.4s	remaining: 11.6s
495:	learn: 0.1255040	total: 11.4s	remaining: 11.6s
496:	learn: 0.1254679	total: 11.4s	remaining: 11.6s
497:	learn: 0.1254153	total: 11.5s	remaining: 11.5s
498:	learn: 0.1253773	total: 11.5s	remaining: 11.5s
499:	learn: 0.1253544	total: 11.5s	remaining: 11.5s
500:	learn: 0.1253171	total: 11.5s	remaining: 11.5s
501:	learn: 0.1252790	total: 11.5s	remaining: 11.4s
502:	learn: 0.1252384	total: 11.6s	remaining: 11.4s
503:	learn: 0.1251935	total: 11.6s	remaining: 11.4s
504:	learn: 0.1251086	total: 11.6s	remaining: 11.4s
505:	learn: 0.1250673	total: 11.6s	remaining: 11.3s
506:	learn: 0.1250318	total: 11.6s	remaining: 11.3s
507:	learn: 

647:	learn: 0.1187600	total: 15.4s	remaining: 8.38s
648:	learn: 0.1187263	total: 15.5s	remaining: 8.37s
649:	learn: 0.1187027	total: 15.5s	remaining: 8.37s
650:	learn: 0.1186596	total: 15.6s	remaining: 8.35s
651:	learn: 0.1186118	total: 15.6s	remaining: 8.34s
652:	learn: 0.1185676	total: 15.7s	remaining: 8.32s
653:	learn: 0.1185373	total: 15.7s	remaining: 8.3s
654:	learn: 0.1184998	total: 15.7s	remaining: 8.29s
655:	learn: 0.1184721	total: 15.8s	remaining: 8.28s
656:	learn: 0.1184434	total: 15.8s	remaining: 8.27s
657:	learn: 0.1184044	total: 15.9s	remaining: 8.25s
658:	learn: 0.1183638	total: 15.9s	remaining: 8.23s
659:	learn: 0.1183337	total: 16s	remaining: 8.22s
660:	learn: 0.1182957	total: 16s	remaining: 8.2s
661:	learn: 0.1182613	total: 16s	remaining: 8.19s
662:	learn: 0.1182294	total: 16.1s	remaining: 8.18s
663:	learn: 0.1182039	total: 16.1s	remaining: 8.16s
664:	learn: 0.1181621	total: 16.2s	remaining: 8.15s
665:	learn: 0.1181292	total: 16.2s	remaining: 8.14s
666:	learn: 0.118073

809:	learn: 0.1129870	total: 20s	remaining: 4.69s
810:	learn: 0.1129380	total: 20s	remaining: 4.66s
811:	learn: 0.1129125	total: 20s	remaining: 4.64s
812:	learn: 0.1128723	total: 20.1s	remaining: 4.62s
813:	learn: 0.1128126	total: 20.1s	remaining: 4.59s
814:	learn: 0.1127740	total: 20.1s	remaining: 4.57s
815:	learn: 0.1127513	total: 20.1s	remaining: 4.54s
816:	learn: 0.1127269	total: 20.2s	remaining: 4.51s
817:	learn: 0.1127111	total: 20.2s	remaining: 4.49s
818:	learn: 0.1126765	total: 20.2s	remaining: 4.46s
819:	learn: 0.1126569	total: 20.2s	remaining: 4.44s
820:	learn: 0.1126184	total: 20.2s	remaining: 4.41s
821:	learn: 0.1125844	total: 20.3s	remaining: 4.39s
822:	learn: 0.1125453	total: 20.3s	remaining: 4.37s
823:	learn: 0.1124961	total: 20.3s	remaining: 4.34s
824:	learn: 0.1124716	total: 20.3s	remaining: 4.32s
825:	learn: 0.1124276	total: 20.4s	remaining: 4.29s
826:	learn: 0.1123973	total: 20.4s	remaining: 4.26s
827:	learn: 0.1123668	total: 20.4s	remaining: 4.24s
828:	learn: 0.1123

970:	learn: 0.1076489	total: 23.8s	remaining: 710ms
971:	learn: 0.1076238	total: 23.8s	remaining: 685ms
972:	learn: 0.1075778	total: 23.8s	remaining: 661ms
973:	learn: 0.1075265	total: 23.8s	remaining: 636ms
974:	learn: 0.1074959	total: 23.9s	remaining: 612ms
975:	learn: 0.1074667	total: 23.9s	remaining: 587ms
976:	learn: 0.1074396	total: 23.9s	remaining: 563ms
977:	learn: 0.1073995	total: 23.9s	remaining: 538ms
978:	learn: 0.1073646	total: 23.9s	remaining: 514ms
979:	learn: 0.1073142	total: 24s	remaining: 489ms
980:	learn: 0.1072733	total: 24s	remaining: 465ms
981:	learn: 0.1072295	total: 24s	remaining: 440ms
982:	learn: 0.1072026	total: 24s	remaining: 416ms
983:	learn: 0.1071922	total: 24.1s	remaining: 391ms
984:	learn: 0.1071416	total: 24.1s	remaining: 367ms
985:	learn: 0.1071238	total: 24.1s	remaining: 342ms
986:	learn: 0.1070960	total: 24.1s	remaining: 318ms
987:	learn: 0.1070792	total: 24.1s	remaining: 293ms
988:	learn: 0.1070400	total: 24.2s	remaining: 269ms
989:	learn: 0.107021

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_ada_cc= GridSearchCV(estimator= cat_ada_cc, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_cat_ada_cc.best_params_)
print("Best Precision Score:", gs_cat_ada_cc.best_score_)

tuned_cat_ada_cc= gs_cat_ada_cc.best_estimator_
y_pred_tuned_cat_ada_cc= tuned_cat_ada_cc.predict(X_test)
y_pred_prob_tuned_cat_ada_cc= tuned_cat_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_ada_cc, labels= tuned_cat_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_cat_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_ada_cc))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.4925349	total: 17.7ms	remaining: 867ms
1:	learn: 0.4111417	total: 37.7ms	remaining: 906ms
2:	learn: 0.3463658	total: 57.4ms	remaining: 899ms
3:	learn: 0.3159072	total: 73.6ms	remaining: 846ms
4:	learn: 0.2979233	total: 89.7ms	remaining: 807ms
5:	learn: 0.2627573	total: 107ms	remaining: 785ms
6:	learn: 0.2552485	total: 123ms	remaining: 754ms
7:	learn: 0.2363669	total: 139ms	remaining: 728ms
8:	learn: 0.2255044	total: 155ms	remaining: 708ms
9:	learn: 0.2149622	total: 171ms	remaining: 683ms
10:	learn: 0.2113658	total: 190ms	remaining: 674ms
11:	learn: 0.2028293	total: 206ms	remaining: 653ms
12:	learn: 0.1919655	total: 225ms	remaining: 641ms
13:	learn: 0.1896358	total: 247ms	remaining: 636ms
14:	learn: 0.1819034	total: 265ms	remaining: 618ms
15:	learn: 0.1746414	total: 282ms	remaining: 600ms
16:	learn: 0.1691302	total: 305ms	remaining: 591ms
17:	learn: 0.1681246	total: 325ms	remaining: 577ms
18:	learn: 0.1669925	tota

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_ada_cc, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_ada_cc, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_ada_cc, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_ada_cc, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_ada_cc, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_ada_cc, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_ada_cc, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_ada_cc, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_ada_cc, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_ada_cc, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_ada_cc, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_ada_cc, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_ada_cc, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_ada_cc, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_ada_cc, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_ada_cc, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_ada_cc, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_ada_cc, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_ada_cc, zero_division= 0))

Logistic Regression Precision: 0.25636553889082664
Decision Tree Precision: 0.23716153127917833
Tuned Decision Tree Precision: 0.30612244897959184
KNeighborsClassifier Precision: 0.218878812860676
Tuned KNeighborsClassifier Precision: 0.25
GaussianNB Precision: 0.1689291101055807
SVM Precision: 0.15121951219512195
Random Forest Precision: 0.41233766233766234
Tuned Random Forest Precision: 0.4412811387900356
AdaBoost Precision: 0.3771266540642722
Tuned AdaBoost Precision: 0.47126436781609193
GradientBoosting Precision: 0.4133522727272727
Tuned GradientBoosting Precision: 0.48476454293628807
XGB Precision: 0.46096654275092935
Tuned XGB Precision: 0.47368421052631576
LGBM Precision: 0.5036496350364964
Tuned LGBM Precision: 0.5265151515151515
CatBoost Precision: 0.4980544747081712
Tuned CatBoost Precision: 0.47750865051903113


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_ada_cc, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_ada_cc, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_ada_cc, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_ada_cc, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_ada_cc, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_ada_cc, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_ada_cc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_ada_cc, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_ada_cc, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_ada_cc, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_ada_cc, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_ada_cc, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_ada_cc, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_ada_cc, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_ada_cc, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_ada_cc, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_ada_cc, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_ada_cc, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_ada_cc, zero_division= 0)
                  }

ada_cc_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
ada_cc_precision= ada_cc_precision.sort_values(by= 'Precision Score', ascending=False)
print(ada_cc_precision)

                                    Model  Precision Score
16                  Tuned LGBM Precision:         0.526515
15                        LGBM Precision:         0.503650
17                    CatBoost Precision:         0.498054
12      Tuned GradientBoosting Precision:         0.484765
18              Tuned CatBoost Precision:         0.477509
14                   Tuned XGB Precision:         0.473684
10              Tuned AdaBoost Precision:         0.471264
13                         XGB Precision:         0.460967
8          Tuned Random Forest Precision:         0.441281
11            GradientBoosting Precision:         0.413352
7                Random Forest Precision:         0.412338
9                     AdaBoost Precision:         0.377127
2          Tuned Decision Tree Precision:         0.306122
0          Logistic Regression Precision:         0.256366
4   Tuned KNeighborsClassifier Precision:         0.250000
1                Decision Tree Precision:         0.2371

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_ada_cc),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_ada_cc),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_ada_cc),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_ada_cc),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_ada_cc),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_ada_cc),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_ada_cc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_ada_cc),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_ada_cc),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_ada_cc),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_ada_cc),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_ada_cc),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_ada_cc),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_ada_cc),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_ada_cc),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_ada_cc),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_ada_cc),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_ada_cc),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_ada_cc)
                  }

ada_cc_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
ada_cc_accuracy= ada_cc_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(ada_cc_accuracy)

                                   Model  Accuracy Score
16                  Tuned LGBM Accuracy:        0.904762
15                        LGBM Accuracy:        0.903489
17                    CatBoost Accuracy:        0.903171
12      Tuned GradientBoosting Accuracy:        0.902111
18              Tuned CatBoost Accuracy:        0.901898
13                         XGB Accuracy:        0.901050
14                   Tuned XGB Accuracy:        0.900944
10              Tuned AdaBoost Accuracy:        0.900095
8          Tuned Random Forest Accuracy:        0.899777
7                Random Forest Accuracy:        0.897550
11            GradientBoosting Accuracy:        0.890338
6                          SVM Accuracy:        0.888111
9                     AdaBoost Accuracy:        0.875703
2          Tuned Decision Tree Accuracy:        0.875066
4   Tuned KNeighborsClassifier Accuracy:        0.865733
1                Decision Tree Accuracy:        0.843568
3         KNeighborsClassifier 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_ada_cc),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_ada_cc),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_ada_cc),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_ada_cc),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_ada_cc),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_ada_cc),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_ada_cc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_ada_cc),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_ada_cc),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_ada_cc),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_ada_cc),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_ada_cc),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_ada_cc),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_ada_cc),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_ada_cc),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_ada_cc),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_ada_cc),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_ada_cc),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_ada_cc)
                  }

ada_cc_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
ada_cc_auc= ada_cc_auc.sort_values(by= 'AUC Score', ascending=False)
print(ada_cc_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.851996
10              Tuned AdaBoost AUC:   0.847504
12      Tuned GradientBoosting AUC:   0.846727
15                        LGBM AUC:   0.846092
18              Tuned CatBoost AUC:   0.845521
11            GradientBoosting AUC:   0.844486
16                  Tuned LGBM AUC:   0.844171
14                   Tuned XGB AUC:   0.843687
9                     AdaBoost AUC:   0.842793
17                    CatBoost AUC:   0.841071
8          Tuned Random Forest AUC:   0.833102
13                         XGB AUC:   0.832070
7                Random Forest AUC:   0.821017
5                   GaussianNB AUC:   0.798326
3         KNeighborsClassifier AUC:   0.738807
2          Tuned Decision Tree AUC:   0.717463
6                          SVM AUC:   0.697508
4   Tuned KNeighborsClassifier AUC:   0.620981
1                Decision Tree AUC:   0.590741
